In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
from bokeh.plotting import figure, output_file, output_notebook, show, save
from bokeh.models import FuncTickFormatter, FixedTicker, NumeralTickFormatter
from bokeh.charts import Bar, Donut

# $ echo 'cvodb2-01:5432:cvodb:cadcuws:bc5e755f2be8e8' >> ~/.pgpass && chmod 600 .pgpass
# $ echo 'cvodb1:5432:cvodb:cadcuws:bc5e755f2be8e8' >> ~/.pgpass
engine = create_engine('postgresql://cadcuws@cvodb2-01:5432/cvodb')
engine_old = create_engine('postgresql://cadcuws@cvodb1:5432/cvodb')

In [30]:
query = "SELECT\
    COUNT(1) AS Tot_q,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.observationID) =%' THEN 1 END) AS Observation_ID,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_pi) LIKE%' THEN 1 END) AS PI_name,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_id) LIKE%' THEN 1 END) AS proposal_id,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_title) LIKE%' THEN 1 END) AS proposal_title,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.proposal_keywords) LIKE%' THEN 1 END) AS Propsal_keyword,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.dataRelease >=% AND Plane.dataRelease <=%' THEN 1 END) AS Data_Rlease_Date,\
    COUNT(CASE WHEN jd.value LIKE '%WHERE  ( Plane.dataRelease <=%' THEN 1 END) AS Data_Rlease_Date_public,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.intent) = ''calibration''%' THEN 1 END) AS Calibration_only,\
    COUNT(CASE WHEN jd.value LIKE '%lower(Observation.intent) = ''science''%' THEN 1 END) AS Science_only,\
    \
    COUNT(CASE WHEN jd.value LIKE '%INTERSECTS( CIRCLE(''ICRS'',%), Plane.position_bounds ) = 1%' THEN 1 END) AS Target,\
    COUNT(CASE WHEN jd.value LIKE '%TAP_UPLOAD.search_upload as f on INTERSECTS(POINT(''ICRS'', f.ra, f.dec)%' THEN 1 END) AS target_upload,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.position_sampleSize >=%AND  Plane.position_sampleSize <=%' THEN 1 END) AS pixel_scale,\
    \
    COUNT(CASE WHEN jd.value LIKE '%INTERSECTS( INTERVAL(%), Plane.time_bounds ) = 1%' THEN 1 END) AS observation_date,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.time_exposure =%' THEN 1 END) AS Integration_time,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.time_bounds_width =%' THEN 1 END) AS time_span,\
    \
    COUNT(CASE WHEN jd.value LIKE '%INTERSECTS( INTERVAL(%), Plane.energy_bounds ) = 1%' THEN 1 END) AS Spactral_Coverage,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_sampleSize >%AND  Plane.energy_sampleSize <%' THEN 1 END) AS Spactral_Sampling,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_resolvingPower >%' THEN 1 END) AS Resolving_power,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_bounds_width >%AND  Plane.energy_bounds_width <%' THEN 1 END) AS Bandpass_width,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_restwav >%AND  Plane.energy_restwav <%' THEN 1 END) AS rest_frame_energy,\
    \
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_emBand =%' THEN 1 END) AS band,\
    COUNT(CASE WHEN jd.value LIKE '%Observation.collection =%' THEN 1 END) AS Collection,\
    COUNT(CASE WHEN jd.value LIKE '%Observation.instrument_name =%' THEN 1 END) AS Instrument,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.energy_bandpassName =%' THEN 1 END) AS Filter,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.calibrationLevel =%' THEN 1 END) AS Calibration_level,\
    COUNT(CASE WHEN jd.value LIKE '%Plane.dataProductType =%' THEN 1 END) AS Data_type,\
    COUNT(CASE WHEN jd.value LIKE '%Observation.type =%' THEN 1 END) AS Observation_type\
    FROM uws.Job job INNER join uws.JobDetail jd ON job.jobid = jd.jobid\
    WHERE job.requestpath = '/tap/sync'\
    AND jd.name = 'QUERY'\
    "
df_summ = pd.read_sql_query(text(query), con = engine).T
df_summ_old = pd.read_sql_query(text(query), con = engine_old).T
    
print(df_summ_old)
print(df_summ)

df_sum = df_summ.add(df_summ_old)
del df_summ_old, df_summ

                               0
tot_q                    9127629
observation_id             13442
pi_name                    16123
proposal_id                19213
proposal_title               939
propsal_keyword              572
data_rlease_date             136
data_rlease_date_public       64
calibration_only             554
science_only              299019
target                   1159863
target_upload               1531
pixel_scale                  149
observation_date            4191
integration_time              70
time_span                     95
spactral_coverage            360
spactral_sampling             23
resolving_power               60
bandpass_width                28
rest_frame_energy            276
band                      299528
collection                374127
instrument                370223
filter                     12761
calibration_level         311622
data_type                 296896
observation_type          295633
                               0
tot_q     

In [31]:
tot = df_sum.at["tot_q", 0]
df_sum = df_sum / tot
p = figure(width = 900, title = "Usage of each Field")
df_sum = df_sum.sort_values(0)
translation = {'spactral_sampling': 'Spectral Sampling', 'bandpass_width': 'Bandpass Width', 'resolving_power': 'Resolving Power', 'integration_time': 'Integration Time', 'time_span': 'Time Span', 'data_rlease_date': 'Data Release Date', 'pixel_scale': 'Pixel Scale', 'rest_frame_energy': 'Rest-frame Energy', 'spactral_coverage': 'Spectral Coverage', 'calibration_only': 'Proposal Keyword', 'propsal_keyword': 'Calibration Data Only', 'proposal_title': 'Proposal Title', 'target_upload': 'Target: File Upload', 'observation_date': 'Observation Date', 'filter': 'Observation ID', 'observation_id': 'Filter', 'pi_name': 'P.I. Name', 'proposal_id': 'Proposal ID', 'band': 'Band', 'calibration_level': 'Calibration Level', 'instrument': 'Instrument', 'observation_type': 'Observation Type', 'data_type': 'Science Data Only', 'science_only': 'Data Type', 'collection': 'Collection', 'target': 'Target', 'data_rlease_date_public': 'Public Data Release Date'}
y = [ _ for _ in range(1, len(df_sum))]
d = dict(zip(y, [translation[x] for x in df_sum.index.values[:-1]]))
output_notebook()
p.hbar(y = y, right = df_sum[0].values[:-1], height = 0.5, left = 0)
p.xaxis.axis_label = ("Percentage (Total Number of Queries %i)" % tot)
p.yaxis.axis_label = "Fields"
p.yaxis[0].ticker = FixedTicker(ticks = y)
p.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p.xaxis[0].formatter = NumeralTickFormatter(format = "0.%")
show(p)
del p

Loading BokehJS ...

In [22]:
query = "SELECT date_trunc('month', job.starttime::DATE) AS starttime, remoteip, count(*)\
                        FROM uws.job job INNER JOIN uws.jobdetail jd ON job.jobid = jd.jobid\
                        WHERE job.requestpath = '/tap/sync'\
                        AND jd.name = 'QUERY'\
                        GROUP BY 1, 2\
                        ORDER BY 1, 3 DESC\
                        "
df_perwk_perip_new = pd.read_sql_query(text(query),con=engine)
df_perwk_perip_old = pd.read_sql_query(text(query),con=engine_old)

df_perwk_perip = pd.concat([df_perwk_perip_old, df_perwk_perip_new])
del df_perwk_perip_old, df_perwk_perip_new

In [23]:
num_q_perweek = df_perwk_perip.assign(starttime = df_perwk_perip.starttime.astype("str").apply(lambda x: x.split(" ")[0]))

x = [ _ for _ in range(len(df_perwk_perip["count"].values)) ]
y = list(df_perwk_perip["count"].values)

p1 = Bar(num_q_perweek, "starttime", "count", legend = False, plot_width = 900,\
        xlabel = 'Date (month)', ylabel = "Counts", title = "Numer of Queries per Week", yscale = "log")

show(p1)
del num_q_perweek, p1

In [24]:
tmp = df_perwk_perip[["count", "remoteip"]].groupby("remoteip").sum().sort_values("count")
num_q_perip_top10 = tmp.tail(9)
num_q_perip_rest = tmp[0:-9]
tmp = sum(num_q_perip_rest.values)
num_q_perip_rest = pd.DataFrame(tmp, index = ["others"], columns = ["count"])
num_q_perip = num_q_perip_rest.append(num_q_perip_top10)

del tmp, num_q_perip_rest, num_q_perip_top10

p2 = figure(width = 900, title = "Number of Queries submitted by IP")
y = [ _ for _ in range(len(num_q_perip))]
d = dict(zip(y, list(num_q_perip.index.values)))
p2.hbar(y = y, right = list(num_q_perip["count"].values / tot), height = 0.5, left = 0)
p2.xaxis.axis_label = ("Percentage (Total Number of Queries %i)" % tot)
p2.yaxis.axis_label = "IP"
p2.yaxis[0].ticker = FixedTicker(ticks = y)
p2.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p2.xaxis[0].formatter = NumeralTickFormatter(format = "0.%")
show(p2)
del num_q_perip, p2

In [25]:
num_q_perip_201612 = df_perwk_perip.assign(starttime = df_perwk_perip.starttime.astype("str")\
                        .apply(lambda x: x.split(" ")[0]))
num_q_perip_201612 = num_q_perip_201612[num_q_perip_201612.starttime == "2016-12-01"][["remoteip", "count"]]\
                        .sort_values("count").tail(10)
p3 = figure(width = 900, title = "Number of Queries submitted in 2016-12 by IP (top 10)")
y = [ _ for _ in range(len(num_q_perip_201612))]
d = dict(zip(y, list(num_q_perip_201612["remoteip"].values)))
p3.hbar(y = y, right = num_q_perip_201612["count"].values, height = 0.5, left = 0)
p3.xaxis.axis_label = "Counts"
p3.yaxis.axis_label = "IP"
p3.yaxis[0].ticker = FixedTicker(ticks = y)
p3.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
show(p3)
del num_q_perip_201612, p3

In [9]:
query = "SELECT AVG(count) FROM (\
    SELECT count(*)\
    FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid\
    WHERE (job.requestpath = '/tap/sync' \
    AND jd.name = 'QUERY' )\
    GROUP BY job.starttime::DATE\
    ) AS avg_q_perday\
    "
df_QperDay_new = pd.read_sql_query(text(query), con = engine)
df_QperDay_old = pd.read_sql_query(text(query), con = engine_old)
print("avg query per day: %i" % (df_QperDay_new.at[0,"avg"] + df_QperDay_old.at[0,"avg"]))
del df_QperDay_new, df_QperDay_old

avg query per day: 38158


In [14]:
#import ipaddress as ip
query = "SELECT ( \
CASE \
WHEN ( job.remoteip ::inet BETWEEN '132.246.0.0'::inet AND '132.246.255.255'::inet) THEN \
    CASE \
    WHEN ((job.remoteip ::inet BETWEEN '132.246.194.0'::inet AND '132.246.194.255'::inet) \
            OR (job.remoteip ::inet BETWEEN '132.246.195.0'::inet AND '132.246.195.255'::inet) \
            OR (job.remoteip ::inet BETWEEN '132.246.217.0'::inet AND '132.246.217.255'::inet)) THEN 'CADC' \
    ELSE 'NRC w/o CADC' \
    END \
WHEN (job.remoteip ::inet BETWEEN '206.12.0.0'::inet AND '206.12.255.255'::inet) THEN 'Compute Canada' \
WHEN (job.remoteip ::inet BETWEEN '192.168.0.0'::inet AND '192.168.255.255'::inet) THEN 'CADC' \
ELSE 'Others' \
END \
) AS domain, job.remoteip \
FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid \
WHERE (job.requestpath = '/tap/sync' AND jd.name = 'QUERY') \
    "
df_domain_new = pd.read_sql_query(text(query), con = engine)
df_domain_old = pd.read_sql_query(text(query), con = engine_old)
df_domain = pd.concat([df_domain_old, df_domain_new])
df_domain = df_domain.assign(event = 1)
del df_domain_new, df_domain_old

In [11]:
grped_domain = df_domain.groupby("domain").count().reset_index()
output_notebook()
p4 = Donut(grped_domain, label = 'domain', values = 'event', plot_width = 600, plot_height = 600, \
           title = "Queries Percentage Submitted by Domains")
show(p4)
del p4

Loading BokehJS ...

In [15]:
df_domain2 = df_domain.groupby(["domain", "remoteip"]).count().reset_index()
print("total unique ip in others %i" % len(df_domain2[df_domain2.domain == "Others"]))
tmp1 = df_domain2[df_domain2.domain == "Compute Canada"].sort_values("event").tail(4)
tmp2 = df_domain2[df_domain2.domain == "CADC"].sort_values("event").tail(4)
tmp3 = df_domain2[df_domain2.domain == "Others"].sort_values("event").tail(4)
tmp4 = df_domain2[df_domain2.domain == "NRC w/o CADC"].sort_values("event").tail(1)
df_domain2 = pd.concat([tmp1, tmp2, tmp3, tmp4])
df_domain2 = df_domain2.sort_values(["domain","event"])
print(df_domain2)
output_notebook()
p5 = Donut(df_domain2, label = ["domain", "remoteip"], values = "event", plot_width = 900, plot_height = 900, \
           title = "Queries Percentage Submitted by Domains")
show(p5)
del p5, df_domain2, tmp1, tmp2, tmp3, tmp4

total unique ip in others 16700
               domain        remoteip    event
89               CADC   132.246.217.5   873578
88               CADC   132.246.217.4   907827
91               CADC   132.246.217.7   989462
90               CADC   132.246.217.6  1028932
99     Compute Canada    206.12.48.81   177826
96     Compute Canada    206.12.26.30  1096979
100    Compute Canada    206.12.48.84  1318145
108    Compute Canada   206.12.59.166  2043382
134      NRC w/o CADC  132.246.192.47  3330152
15000          Others  79.116.245.137    43694
15654          Others    86.127.216.4    57897
9510           Others   194.35.74.243   170155
1799           Others  128.171.90.200  2817548


Loading BokehJS ...

In [16]:
query = "\
SELECT ip, SUBSTRING(query_body, 'Observation\.collection\ \=\ ''([A-Z]+)''') \
FROM ( \
    SELECT ( \
    CASE  \
        WHEN ( job.remoteip ::inet BETWEEN '132.246.0.0'::inet AND '132.246.255.255'::inet) THEN \
            CASE \
                WHEN ((job.remoteip ::inet BETWEEN '132.246.194.0'::inet AND '132.246.194.255'::inet) OR (job.remoteip ::inet BETWEEN '132.246.195.0'::inet AND '132.246.195.255'::inet) OR (job.remoteip ::inet BETWEEN '132.246.217.0'::inet AND '132.246.217.255'::inet)) THEN 'CADC'  \
                ELSE 'NRC' \
            END \
        WHEN (job.remoteip ::inet BETWEEN '206.12.0.0'::inet AND '206.12.255.255'::inet) THEN 'Compute Canada' \
        WHEN (job.remoteip ::inet BETWEEN '192.168.0.0'::inet AND '192.168.255.255'::inet) THEN 'Private' \
        ELSE 'Others'  \
    END \
    ) AS domain, jd.value AS query_body, job.remoteip AS ip \
    FROM uws.Job job INNER JOIN uws.JobDetail jd ON job.jobid = jd.jobid  \
    WHERE (job.requestpath = '/tap/sync' AND jd.name = 'QUERY') \
) \
AS Collection_value \
WHERE (domain = 'Others') \
"
df_Others_new = pd.read_sql_query(text(query), con = engine)
df_Others_old = pd.read_sql_query(text(query), con = engine_old)

df_Others = pd.concat([df_Others_old, df_Others_new])

del df_Others_new, df_Others_old

In [394]:
# this cell generates ip2dom.json, a look-up json file for all the ips that are outside NRC/CADC and CC

# import json
# import socket
# from publicsuffix import fetch, PublicSuffixList

# psl_file = fetch()
# psl = PublicSuffixList(psl_file)

# tmp = df_Others["ip"].value_counts()

# dom = []
# l = list(tmp.index.values)
# for i in range(len(l)):
#     tmp.append(psl.get_public_suffix(socket.getfqdn(x)))
     
# x = dict(zip(tmp.index.values, dom))
# with open("ip2dom.json", "w") as f:
#     json.dump(x, f);
# del x, tmp, dom

In [29]:
#import geoip2.database
import json
import socket
from publicsuffix import fetch, PublicSuffixList

#reader = geoip2.database.Reader('/Users/will/GeoLite2-City.mmdb')

# please see above cell to generate ip2dom.json
with open("ip2dom.json", "r") as fin: ip2dom = json.load(fin);
# update ip2dom.json if new ip shows ip
psl_file = fetch()
psl = PublicSuffixList(psl_file)    
for x in df_Others["ip"].values:
    try:
        ip2dom[x]
    except KeyError:
        ip2dom[x] = psl.get_public_suffix(socket.getfqdn(x))
        
with open("ip2dom.json", "w") as fout:
    json.dump(ip2dom, fout)
    
df_Others["host"] = df_Others["ip"].apply(lambda x: ip2dom[x])
# drop googlebot crawler
df_Others = df_Others[df_Others.host != "googlebot.com"]
tot_others = len(df_Others)
s1 = df_Others["host"].value_counts(ascending = True) / tot_others
s1_top10 = s1.tail(7)
s1_rest = s1[0:-7]
others = sum(s1_rest.values)
s1_rest = pd.Series(data = others, index = ["others"])
s1 = s1_rest.append(s1_top10)
output_notebook()
p6 = figure(width = 1000, height = 800, title = "Number of Queries submitted by Non-NRC/CADC Domains")
y = [ _ for _ in range(len(s1))]
d = dict(zip(y, list(s1.index.values)))
p6.hbar(y = y, right = s1.values, height = 0.5, left = 0)
p6.xaxis.axis_label = ("Percentage (Total Number of Queries Submitted by Non-NRC/CADC Domains: %i)" % tot_others)
p6.yaxis.axis_label = "Domain"
p6.yaxis[0].ticker = FixedTicker(ticks = y)
p6.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p6.xaxis[0].formatter = NumeralTickFormatter(format = "0.%")
show(p6)
#save(p6, "NumQueryByNonNRC-CADCDom.html")
del p6, s1

Loading BokehJS ...

In [21]:
s2 = df_Others["substring"].value_counts( ascending = True, dropna = False) / tot_others
#print(s2)
output_notebook()
p7 = figure(width = 900, height = 900, title = \
            'Most Selected "Collection" for Non-NRC/CADC Domains \
({0:.2f}% of Queries Selected "Collection")'.format(100 * (1 - s2[-1])))
y = [ _ for _ in range(len(s2) - 1)]
d = dict(zip(y, list(s2.index.values)[:-1]))
p7.hbar(y = y, right = s2.values[:-1], height = 0.5, left = 0)
p7.xaxis.axis_label = ("Percentage (Total Number of Queries Submitted by Non-NRC/CADC Domains: %i)" % tot_others)
p7.yaxis.axis_label = 'Collection'
p7.yaxis[0].ticker = FixedTicker(ticks = y)
p7.yaxis[0].formatter = FuncTickFormatter(code = """dic = """ + str(d) + """
     return dic[tick]""")
p7.xaxis[0].formatter = NumeralTickFormatter(format = "0.00%")
show(p7)
del p7, s2

Loading BokehJS ...